# Machine Learning: A Simple Demonstration

There is clearly much interest and buzz around machine learning right now. Machine learning has incredible power to help organisations predict who will buy, lie or die, but increasingly the term is being used without clear articulation of the problems it can solve or quantifying the benefits compared to non-machine learning techniques. The term appears to be misunderstood and misused. So, what is machine learning really?

## TL;DR
- Machine learning discovers patterns within data but does not identify 'rules' that define those patterns.
- A machine learning algorithm needs to be trained on examples where the outcome is known.
- It does not output 'rules' that explain its decisions.
- It generalises the relationships within data.
- It cannot be used reliably on data outside the range it has been trained.

## Basic Definition
Put simply, it is a collection of techniques that aim to uncover patterns in data. It is particularly useful when it is impossible or impractical to define a set of rules. It could be that the relationships within the data are too complex or we simply have no idea how to get started developing rules.

For example recognising damage in cars from photos. It would be impractical to break this down into _if...then_ rules on individual image pixels. Instead we can show a machine learning algorithm a 'labelled' dataset (the training set) where photos have been labelled in some way and the algorithm learns the pattern (hence machine learning) in order to make a prediction on an unseen photo. This is termed 'supervised learning' and we can often train the machine to be better than the most experienced human (and significantly faster). The caveat is that we must have a training set with enough examples of all scenarios.

You might hear machine learning called a 'black-box' method. This means that we do not know what rules it has discovered; the output of machine learning is __not__ a rules engine that can be coded in SQL or similar. Rules are different from model explainability: more on explainability below.

Machine learning is good at generalising relationships within data, but only on data in ranges that it has seen before. It does not discover underlying 'rules' in data (assuming rules even exist). The behaviour of a machine learning model can mimic underlying rules but it is __not__ a 'rules engine'.

## Example
The following demonstration illustrates what happens when we 'teach' a machine learning algorithm to perform multiplication.

The Python code used in the examples is provided so the interested reader can explore and experiment. This article is also available as a Jupyter notebook at __[SVR Demo](https://github.com/J-Robinson/ML-examples/blob/master/SVR%20Demo.ipynb)__.

First import some packages:

In [1]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format
np.set_printoptions(precision=2)

Let's define some training data. The training data is a set of examples where we know what the output should be. In this example we have a set of training examples where $X_{1} \times X_{2} = Y$. Note that all our examples are integers in the range 0 to 10, so our algorithm will only be trained in this narrow range.

In [2]:
X = [[0, 0], [1, 2], [2, 3], [5, 9], [9, 5], [1, 4], [6, 2], [2, 6], [8, 10], [10, 10]]
y = [    0,      2,      6,     45,     45,      4,     12,     12,      80,      100]

Now train our algorithm. In this case we are using support vector machine regression (SVR), but other algorithms could be used such as gradient boosted trees, random forests, etc. so long as it is non-linear. SVR requires a few parameters to be optimised, so we include a grid search to try all combinations and give us the best parameters.

In [3]:
%%capture

# Create an SVR object
svm = svm.SVR(C=65000, epsilon=0.001)

# Define parameters to be tested
parameters = [{'kernel': ['rbf'], 
               'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
               'C': [1, 10, 100, 1000, 10000, 50000, 100000, 500000, 1000000]}]

# Perform a grid search to tune the parameters
clf = GridSearchCV(svm, parameters, cv=5)

# Finally train the model with the training set
clf.fit(X, y) 

The next step is test some in-sample data. These are examples that were used during the training process.

In [4]:
print(f"2 x 3 is {clf.predict([[2,3]])[0]:.2f}")
print(f"5 x 9 is {clf.predict([[5,9]])[0]:.2f}")
print(f"6 x 2 is {clf.predict([[6,2]])[0]:.2f}")
print(f"10 x 10 is {clf.predict([[10,10]])[0]:.2f}")
print(f"8 x 10 is {clf.predict([[8,10]])[0]:.2f}")

2 x 3 is 5.90
5 x 9 is 45.00
6 x 2 is 12.00
10 x 10 is 100.00
8 x 10 is 80.00


Here we can see how the algorithm has generalised. It has got most predictions correct with a slight error in $2 \times 3$. It is essentially saying $2 \times 3$ is about 6. It has not discovered the _rule_ of multiplication. It is not as accurate as a calculator for this simple task.

We will now see how well our algorithm has learned by asking it to predict multiplication on out-of-sample data, i.e. data it has not seen before, but within the range it was trained.

In [5]:
print(f"1 x 8 is {clf.predict([[1,8]])[0]:.2f}")
print(f"6 x 6 is {clf.predict([[6,6]])[0]:.2f}")
print(f"4 x 6 is {clf.predict([[4,6]])[0]:.2f}")
print(f"4 x 5 is {clf.predict([[4,5]])[0]:.2f}")
print(f"9 x 8 is {clf.predict([[9,8]])[0]:.2f}")

1 x 8 is 8.77
6 x 6 is 35.73
4 x 6 is 23.74
4 x 5 is 19.68
9 x 8 is 72.22


It is not as good as the in-sample test. This is an expected outcome when machine learning is used for this type of problem. It is has given approximate answers that are close to the correct values. Let's see how well it performs on data outside the range of the training set:

In [6]:
print(f"10 x 12 is {clf.predict([[10,12]])[0]:.2f}")
print(f"15 x 15 is {clf.predict([[15,15]])[0]:.2f}")
print(f"20 x 10 is {clf.predict([[20,10]])[0]:.2f}")
print(f"100 x 100 is {clf.predict([[100,100]])[0]:.2f}")
print(f"-3 x 5 is {clf.predict([[-3,5]])[0]:.2f}")

10 x 12 is 118.82
15 x 15 is 211.06
20 x 10 is 190.13
100 x 100 is 46.88
-3 x 5 is -13.53


Clearly the algorithm cannot perform multiplication reliably on numbers outside the range $0$ to $10$. The further away we are from the range of training data, the worse the prediction. The extreme example of $100 \times 100$ being $\approx46$ is not even vaguely correct. Interestingly it didn't do too badly on $-3 \times 5$ being $\approx-14$ despite not being trained on negative numbers.

The takeaways from this simple example are:

- Machine learning discovers patterns within data but does not identify 'rules' that define those patterns.
- A machine learning algorithm needs to be trained on examples where the outcome is known.
- It does not output 'rules' that explain its decisions.
- It generalises the relationships within data.
- It cannot be used reliably on data outside the range it has been trained.

There are techniques to improve these limitations such as scaling the inputs (Z-Score, Min-Max, etc.) and transformation of the dependent variable through methods such as log-normal, taking the first order derivative, etc.

## Why would we use machine learning?
Mostly because of its ability to generalise patterns with data. This example was trivial, but real world problems are not and we do not know what the underlying rules are, or even that they exist. For example, a credit decision can take hundreds or thousands of inputs. The machine learning algorithm can learn the relationship between these inputs and examples of high and low risk.

It is important to use inputs that are within the range that the algorithm was trained. If we've trained a credit risk model on data only for customers who had lending up to $\$500,000$, it must not be used to predict credit worthiness for customers wanting to take out lending of $\$1,000,000$. When deploying a model into production, this is something that needs to be taken into account.

## Model Explainability
If a model needs explainability, some algorithms are able to return a _significant feature_ analysis. This is a list of inputs ranked from most important to least important. This tells us what the algorithm considers to be important and can be used as the basis of an explanation on its decisions. This does have limitations however. Just because a feature is significant does not give us the context of why it is significant. For example an algorithm predicting employment may tell us that a significant factor is whether someone buys a train ticket regularly or not. That is what the data says. It does not tell us why this is significant, just that it is. We would then use this to formulate hypotheses and conduct separate experiments to verify these hypotheses.

Another example is that a machine learning algorithm designed to predict daily stock movements may return a 21 period moving average as the most significant factor. Why is this? Why 21 periods and not 20 or 75? What is the underlying macro/micro economic driver for this specific period of moving average?

## Conclusion
Machine learning is a powerful set of techniques and methods. Like any tool it must be used properly and appropriately. The analyst and data scientist must be aware of any limitations of the technique they are using and be sure guard rails and checks are in place during model execution in a production environment. 